In [1]:
#import statements
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.pipeline import  QAPipeline
from cdqa.utils.download import download_model

In [52]:
#Download Model
download_model(model='bert-squad_1.1', dir='./model')


bert_qa.joblib already downloaded


In [53]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [56]:
df = pdf_converter(directory_path='./docs')
df.head()

title  \
0  Employee-Referral-Programme   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [57]:
pd.set_option('display.max_colwidth', -1)

e:\entity_extraction_combined_model\venv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [58]:
cdqa_pipeline = QAPipeline(reader='./model/bert_qa.joblib', max_df=1.0)
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(learning_rate=3e-05, num_train_epochs=2,
                         output_dir='model', train_batch_size=12),
           retriever=BM25Retriever(max_df=1.0))

In [59]:
query="Referrer Employee referring to "
prediction = cdqa_pipeline.predict(query)
prediction

e:\entity_extraction_combined_model\venv\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass msg=idf vector is not fitted as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[{'title': 'Employee-Referral-Programme', 'paragraphs': [{'context': 'Referrer Employee referring the potential candidate ', 'qas': [{'answers': [], 'question': 'Referrer Employee referring to ', 'id': '62fbe4b3-9ee0-477d-b448-a1072a8a0ad8', 'retriever_score': array([6.8050519])}]}]}, {'title': 'Employee-Referral-Programme', 'paragraphs': [{'context': 'Referral Bonus Payment made to the referrer  4.0   Guidelines for Referrals  4.1 Eligibility for referring candidates • All employees of Techversant are eligible to refer candidates under the employee referral program. • Referrer should not be involved in the hiring process of the referred candidate (viz. prescreening, interview and offer release). • All those directly involved in the hiring process for the referral positions will not be part of this programme.  ‘Confidential & Proprietary’ ‘Property of Techversant’  ', 'qas': [{'answers': [], 'question': 'Referrer Employee referring to ', 'id': '04322059-ca93-40f4-a2f6-e819614bfadb', 'r

('Employee referring the potential',
 'Employee-Referral-Programme',
 'Referrer Employee referring the potential candidate ',
 2.797833302568006)

In [16]:
import requests
import json
import torch
import os
from tqdm import tqdm

In [17]:
# Load the training dataset and take a look at it
with open('C:\\Users\\JaiGatiri\\Documents\\cdqa-pred\\annotated_data\\cdqa-v1_updated_referral_annot.json', 'rb') as f:
  squad = json.load(f)

In [18]:
squad['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [19]:
# Find the group about Greece
gr = -1
for idx, group in enumerate(squad['data']):
  print(group['title'])
  if group['title'] == 'Employee Referral Programme':
    gr = idx
    print(gr)
    break

Employee Referral Programme
0


In [20]:
# let's check on Greece which is 186th (0-based indexing)
# we can see that we have a context and many questions and answers following
squad['data'][0]

{'title': 'Employee Referral Programme',
 'paragraphs': [{'context': 'Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special c

In [21]:
# and this is the context given for NYC
squad['data'][0]['paragraphs'][3]['context']

'The employee referral process will be as below,the Talent Acquisition Team will declare the open positions which would come under the employee referral program. It will have the referral category mentioning the bonus amount for the referral positions and the date until which the program is open  for the respective referral bonus along with the other relevant position details. Only positions announced under the employee referral bonus programme will be considered for processing. The Referral Bonus Amount will be notified for the open positions, based on the criticality  and priority of the position, availability in the market, the role and designation of the position and the quality of the candidate hired. The  Referral  Bonus  amount  could  range  from  INR  5000  to  INR  50,000  or  an amount  as  decided  by  the  Talent  Acquisition  Team  and  approved  by  the Management based on the open position.All referrals are to be sent to the HR Email id or uploaded to a portal, as notif

In [13]:
def read_data(path):  
  # load the json file
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

In [14]:
train_contexts, train_questions, train_answers = read_data("C:\\Users\\JaiGatiri\\Documents\\cdqa-pred\\annotated_data\\cdqa-v1_updated_referral_annot.json")

In [15]:
# print a random question and answer
print(f'There are {len(train_questions)} questions')
print(train_questions[-54])
print(train_answers[-54])

There are 54 questions
What is the purpose of Employee Referral Programme ?
{'answer_start': 0, 'text': 'Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization.'}


In [82]:
def add_end_idx(answers, contexts):
  for answer, context in zip(answers, contexts):
    gold_text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(gold_text)

    # sometimes squad answers are off by a character or two so we fix this
    if context[start_idx:end_idx] == gold_text:
      answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
      answer['answer_start'] = start_idx - 1
      answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
    elif context[start_idx-2:end_idx-2] == gold_text:
      answer['answer_start'] = start_idx - 2
      answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)

In [83]:
# You can see that now we get the answer_end also
print(train_questions[-54])
print(train_answers[-54])

What is the purpose of Employee Referral Programme ?
{'answer_start': 0, 'text': 'Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization.', 'answer_end': 205}


In [84]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [85]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [86]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 54 context-question pairs


In [87]:
train_encodings['input_ids'][0]

[101,
 3800,
 1998,
 7863,
 1997,
 1996,
 7904,
 6523,
 7941,
 4746,
 2832,
 6254,
 2003,
 2000,
 3073,
 11594,
 4072,
 2000,
 6509,
 2035,
 5126,
 2012,
 6627,
 14028,
 4630,
 2040,
 6523,
 5848,
 17879,
 2000,
 1996,
 3029,
 1012,
 2023,
 2565,
 2003,
 2881,
 2000,
 2393,
 3120,
 5848,
 1010,
 3857,
 8906,
 1010,
 1998,
 2424,
 1996,
 2157,
 5347,
 2005,
 2330,
 4460,
 1998,
 2000,
 10377,
 2256,
 5126,
 2005,
 1996,
 2157,
 6523,
 7941,
 2015,
 1012,
 2023,
 4746,
 2097,
 8627,
 2035,
 2012,
 6627,
 14028,
 4630,
 2000,
 2552,
 2004,
 5848,
 20986,
 2005,
 1996,
 3029,
 1012,
 6627,
 14028,
 4630,
 2003,
 2019,
 5020,
 4495,
 11194,
 2007,
 1037,
 7578,
 14877,
 1012,
 2035,
 3615,
 5347,
 2097,
 2022,
 2445,
 1996,
 2168,
 9584,
 1998,
 2097,
 3413,
 2083,
 1996,
 2511,
 2832,
 1998,
 8853,
 1997,
 1996,
 3029,
 1012,
 3531,
 3602,
 2008,
 2009,
 2003,
 2025,
 1996,
 7848,
 2005,
 2122,
 11594,
 2000,
 6235,
 2296,
 3663,
 1012,
 2968,
 8689,
 2097,
 1013,
 2089,
 2022,
 4072,
 199

In [88]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] purpose and objective of the employee referral programme process document is to provide guidelines necessary to assist all employees at techversant who refer talent profiles to the organization. this program is designed to help source talent, build diversity, and find the right candidates for open positions and to reward our employees for the right referrals. this programme will encourage all at techversant to act as talent ambassadors for the organization. techversant is an equal opportunity employer with a diverse workforce. all referred candidates will be given the same consideration and will pass through the established process and procedures of the organization. please note that it is not the intent for these guidelines to describe every situation. management judgment will / may be necessary in certain situations which are not addressed here, or where special considerations may be required. [SEP] what is the purpose of employee referral programme? [SEP] [PAD] [PAD] [PAD] [P

In [90]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)

In [91]:
train_encodings['start_positions'][:10]

[1, 512, 83, 118, 6, 83, 33, 33, 33, 26]

In [92]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

In [93]:
train_dataset = SQuAD_Dataset(train_encodings)

In [95]:
from torch.utils.data import DataLoader

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [96]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [97]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cpu


In [98]:
from transformers import AdamW

N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

Epoch 5: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:58<00:00, 14.71s/it, loss=2.42]


In [99]:
model_path = 'C:/Users/JaiGatiri/Documents/cdqa-pred/cdQA/squad-2.0-model'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('C:/Users/JaiGatiri/Documents/cdqa-pred/cdQA/squad-2.0-model\\tokenizer_config.json',
 'C:/Users/JaiGatiri/Documents/cdqa-pred/cdQA/squad-2.0-model\\special_tokens_map.json',
 'C:/Users/JaiGatiri/Documents/cdqa-pred/cdQA/squad-2.0-model\\vocab.txt',
 'C:/Users/JaiGatiri/Documents/cdqa-pred/cdQA/squad-2.0-model\\added_tokens.json')

In [100]:
def get_prediction(context, question):
  inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
  outputs = model(**inputs)
  
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 
  
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  
  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)
  def white_space_fix(text):
    return " ".join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match(prediction, truth):
    return bool(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  return round(2 * (prec * rec) / (prec + rec), 2)
  
def question_answer(context, question,answer):
  prediction = get_prediction(context,question)
  em_score = exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f'Question: {question}')
  print(f'Prediction: {prediction}')
  print(f'True Answer: {answer}')
  print(f'Exact match: {em_score}')
  print(f'F1 score: {f1_score}\n')

In [ ]:
# context = """Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special considerations may be required."""


questions = ["Does the guideline change ?"]

answers = ["Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization."]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

# Fine tune BERT

In [119]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time

In [122]:
# Give the path for train data
path = Path("C:\\Users\\JaiGatiri\\Documents\\cdqa-pred\\annotated_data\\cdqa-v1_updated_referral_annot.json")

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

texts = []
queries = []
answers = []

# Search for each passage, its question and its answer
for group in squad_dict['data']:
    for passage in group['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            question = qa['question']
            for answer in qa['answers']:
                # Store every passage, query and its answer to the lists
                texts.append(context)
                queries.append(question)
                answers.append(answer)

train_texts, train_queries, train_answers = texts, queries, answers

In [123]:
# Give the path for validation data
path = Path("C:\\Users\\JaiGatiri\\Downloads\\test-er.json")

# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

texts = []
queries = []
answers = []

# Search for each passage, its question and its answer
for group in squad_dict['data']:
    for passage in group['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            question = qa['question']
            for answer in qa['answers']:
                # Store every passage, query and its answer to the lists
                texts.append(context)
                queries.append(question)
                answers.append(answer)

val_texts, val_queries, val_answers = texts, queries, answers

In [124]:
print(len(train_texts))
print(len(train_queries))
print(len(train_answers))

54
54
54


In [125]:


print("Passage: ",train_texts[0])  
print("Query: ",train_queries[0])
print("Answer: ",train_answers[0])

Passage:  Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special considerations may be required.
Query:  What is the purpose o

In [126]:
print(len(val_texts))
print(len(val_queries))
print(len(val_answers))

18
18
18


In [135]:
print("Passage: ",val_texts[0])  
print("Query: ",val_queries[0])
print("Answer: ",val_answers[0])

Passage:  This employee referral program  is  developed to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special considerations may be required.
Query:  Does this program help to source talent ?
Answer:  {'answer_start': 0, 'text': 'This employee referral program  is  developed to  help  source  talent, build  diversity,  and  find  the  right  candidates  for

In [136]:
for answer, text in zip(train_answers, train_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters 
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1  
    # When the real answer is more by two characters  
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2    

In [137]:
for answer, text in zip(val_answers, val_texts):
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters 
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1  
    # When the real answer is more by two characters  
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2   

In [138]:
from transformers import AutoTokenizer,AdamW,BertForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, train_queries, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, val_queries, truncation=True, padding=True)

In [139]:
## Find out start_position and end_position in encoded dataset
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []

    for idx in range(len(answers)):
        start_positions.append(encodings.char_to_token(idx, answers[idx]['answer_start']))
        if answers[idx]['answer_end'] is None:
          end_positions.append(encodings.char_to_token(idx, answers[idx]['answer_end']))
        else:
          end_positions.append(encodings.char_to_token(idx, answers[idx]['answer_end'] - 1))
        
        #if None, the answer passage has been truncated due to words > 512 so setting last position as 511
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length-1
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length-1
    
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)


ValueError: char_to_token() is not available when using Python based tokenizers

In [2]:
!pip install seaborn

You should consider upgrading via the 'e:\entity_extraction_combined_model\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [6]:
coqa = pd.read_json("C:\\Users\\JaiGatiri\\Documents\\cdqa-pred\\annotated_data\\cdqa-v1_updated_referral_annot.json")
coqa.head()

,version,data
0,v1.1,"{'title': 'Employee Referral Programme', 'para..."


In [7]:
del coqa["version"]

In [9]:
cols = ["text","question","answer"]

# j = 1
comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
#         temp_list.append(j)
        temp_list.append(row["data"]["paragraphs"])
        temp_list.append(row["data"]["paragraphs"]["qas"][i]["question"])
        temp_list.append(row["data"]["qas"]["answers"][i]["text"])
        comp_list.append(temp_list)
#     j += 1
new_df = pd.DataFrame(comp_list, columns=cols)

KeyError: 'questions'

In [1]:
import json
import os
import re
import string
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# ============================================= PREPARING DATASET ======================================================
class Sample:
    def __init__(self, question, context, start_char_idx=None, answer_text=None, all_answers=None):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False
        self.start_token_idx = -1
        self.end_token_idx = -1

    def preprocess(self):
        context = " ".join(str(self.context).split())
        question = " ".join(str(self.question).split())
        tokenized_context = tokenizer.encode(context)
        tokenized_question = tokenizer.encode(question)
        if self.answer_text is not None:
            answer = " ".join(str(self.answer_text).split())
            end_char_idx = self.start_char_idx + len(answer)
            if end_char_idx >= len(context):
                self.skip = True
                return
            is_char_in_ans = [0] * len(context)
            for idx in range(self.start_char_idx, end_char_idx):
                is_char_in_ans[idx] = 1
            ans_token_idx = []
            for idx, (start, end) in enumerate(tokenized_context.offsets):
                if sum(is_char_in_ans[start:end]) > 0:
                    ans_token_idx.append(idx)
            if len(ans_token_idx) == 0:
                self.skip = True
                return
            self.start_token_idx = ans_token_idx[0]
            self.end_token_idx = ans_token_idx[-1]
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(tokenized_question.ids[1:])
        attention_mask = [1] * len(input_ids)
        padding_length = max_seq_length - len(input_ids)
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:
            self.skip = True
            return
        self.input_word_ids = input_ids
        self.input_type_ids = token_type_ids
        self.input_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets

In [3]:
def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                if "answers" in qa:
                    answer_text = qa["answers"][0]["text"]
                    all_answers = [_["text"] for _ in qa["answers"]]
                    start_char_idx = qa["answers"][0]["answer_start"]
                    squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
                else:
                    squad_eg = Sample(question, context)
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples

In [4]:
def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_word_ids": [],
        "input_type_ids": [],
        "input_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
    x = [dataset_dict["input_word_ids"],
         dataset_dict["input_mask"],
         dataset_dict["input_type_ids"]]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x,y

In [5]:
# =================================================== TRAINING =========================================================


class ValidationCallback(keras.callbacks.Callback):

    def normalize_text(self, text):
        text = text.lower()
        text = "".join(ch for ch in text if ch not in set(string.punctuation))
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        text = re.sub(regex, " ", text)
        text = " ".join(text.split())
        return text

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]
            normalized_pred_ans = self.normalize_text(pred_ans)
            normalized_true_ans = [self.normalize_text(_) for _ in squad_eg.all_answers]
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch + 1}, exact match score={acc:.2f}")

In [6]:
train_path = "C:\\Users\\JaiGatiri\\Documents\\cdqa-pred\\annotated_data\\cdqa-v1_updated_referral_annot.json"
eval_path = "C:\\Users\\JaiGatiri\\Downloads\\test-er.json"
with open(train_path) as f: raw_train_data = json.load(f)
with open(eval_path) as f: raw_eval_data = json.load(f)
max_seq_length = 384
print(raw_train_data)

{'version': 'v1.1', 'data': [{'title': 'Employee Referral Programme', 'paragraphs': [{'context': 'Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addre

In [17]:
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2")
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy().decode("utf-8")
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertWordPieceTokenizer(vocab=vocab_file, lowercase=True)
train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")
eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")
start_logits = layers.Dense(1, name="start_logit", use_bias=False)(sequence_output)
start_logits = layers.Flatten()(start_logits)
end_logits = layers.Dense(1, name="end_logit", use_bias=False)(sequence_output)
end_logits = layers.Flatten()(end_logits)
start_probs = layers.Activation(keras.activations.softmax)(start_logits)
end_probs = layers.Activation(keras.activations.softmax)(end_logits)
model = keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=[start_probs, end_probs])
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer=optimizer, loss=[loss, loss])
model.summary()
model.fit(x_train, y_train, epochs=10, batch_size=8, callbacks=[ValidationCallback(x_eval, y_eval)])
model.save_weights("./weights.h5")

54 training points created.
18 evaluation points created.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                  

In [20]:
data = {"data":
    [
        {"title": "Employee Referral Programme",
         "paragraphs": [
             {
                 "context": "Scope defines the  process will be applicable for all referrals hired through the employee referral program, effective 1st October 2021.It covers all open positions in the Organization that would fall under Employee Referral Programme as announced by the Talent Acquisition Team and as approved by the Management.Special referral schemes catering to critical needs as announced by the Talent Acquisition Team and as approved by the management.",
            
                 "qas": [
                     {"question": "Do yo have special referral schemes ?",
                      "id": "Q1"
                      }
                 ]}]}]}

test_samples = create_squad_examples(data)
x_test, _ = create_inputs_targets(test_samples)
pred_start, pred_end = model.predict(x_test)
for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
    test_sample = test_samples[idx]
    offsets = test_sample.context_token_to_char
    start = np.argmax(start)
    end = np.argmax(end)
    pred_ans = None
    if start >= len(offsets):
        continue
    pred_char_start = offsets[start][0]
    if end < len(offsets):
        pred_ans = test_sample.context[pred_char_start:offsets[end][1]]
    else:
        pred_ans = test_sample.context[pred_char_start:]
    print("Q: " + test_sample.question)
    print("A: " + pred_ans)

Q: Do yo have special referral schemes ?
A:  would fall under Employe


In [21]:
import requests
import json
import torch
import os
from tqdm import tqdm

In [22]:
!pip install transformers

You should consider upgrading via the 'e:\entity_extraction_combined_model\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [37]:
# Load the training dataset and take a look at it
with open("C:\\Users\\JaiGatiri\\Documents\\cdqa-pred\\annotated_data\\cdqa-v1_updated_referral_annot.json", 'rb') as f:
  squad = json.load(f)

In [38]:
# Each 'data' dict has two keys (title and paragraphs)
squad['data'][186].keys()

IndexError: list index out of range

In [25]:
# Find the group about Greece
gr = -1
for idx, group in enumerate(squad['data']):
  print(group['title'])
  if group['title'] == 'Employee Referral Programme':
    gr = idx
    print(gr)
    break

Employee Referral Programme
0


In [28]:
squad['data'][0]

{'title': 'Employee Referral Programme',
 'paragraphs': [{'context': 'Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special c

In [32]:
# and this is the context given for NYC
squad['data'][0]['paragraphs'][0]['context']

'Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special considerations may be required.'

In [5]:
!pip install sentencepiece

You should consider upgrading via the 'e:\entity_extraction_combined_model\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [26]:
%cd C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\BERT-SQuAD-master

C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\BERT-SQuAD-master


In [22]:
torch. __version__

'1.4.0+cpu'

In [15]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
   XLNetConfig,
   XLNetForQuestionAnswering,
   XLNetTokenizer,
   squad_convert_examples_to_features
)

In [31]:
!list

'list' is not recognized as an internal or external command,
operable program or batch file.


In [35]:
!pip install pytorch

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for Pytorch
Failed to build Pytorch
    Running setup.py install for Pytorch: started
    Running setup.py install for Pytorch: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'e:\entity_extraction_combined_model\venv\scripts\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\JAIGAT~1\\AppData\\Local\\Temp\\pip-install-frg0gkyy\\pytorch_9ba77d4bbc9d4ec5a7a238ab70a696c6\\setup.py'"'"'; __file__='"'"'C:\\Users\\JAIGAT~1\\AppData\\Local\\Temp\\pip-install-frg0gkyy\\pytorch_9ba77d4bbc9d4ec5a7a238ab70a696c6\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\JAIGAT~1\AppData\Local\Temp\pip-wheel-2d2t49mf'
       cwd: C:\Users\JAIGAT~1\AppData\Local\Temp\pip-install-frg0gkyy\pytorch_9ba77d4bbc9d4ec5a7a238ab70a696c6\
  Complete output (5 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
 

In [36]:
!pip install transformers

You should consider upgrading via the 'e:\entity_extraction_combined_model\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [39]:
SQUAD_DIR="C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\BERT-SQuAD-master"

In [42]:
python run_squad.py \
  --model_type xlnet \
  --model_name_or_path xlnet-large-cased \
  --do_train \
  --do_lower_case \
  --train_file ${SQUAD_DIR}/all_3docs_annotn_cdqa.json\
  --predict_file ${SQUAD_DIR}/test-er.json \
  --num_train_epochs 3 \
  --learning_rate 3e-5 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --save_steps=10000 \
  --per_gpu_train_batch_size 1 \
  --gradient_accumulation_steps 4 \
  --output_dir ./xlnet_large_squad1_dist_X \

SyntaxError: invalid syntax (Temp/ipykernel_11476/1412944235.py, line 1)

In [43]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
   XLNetConfig,
   XLNetForQuestionAnswering,
   XLNetTokenizer,
   squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = True

if use_own_model:
  model_name_or_path = "C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\BERT-SQuAD-master\\model\\"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# # Setup model
# config_class, model_class, tokenizer_class = (
#    XLNetConfig,
#    XLNetForQuestionAnswering,
#    XLNetTokenizer)
# config = config_class.from_pretrained(model_name_or_path)
# tokenizer = tokenizer_class.from_pretrained(
#     model_name_or_path, do_lower_case=True)
#model = model_class.from_pretrained(model_name_or_path, config=config)

model = torch.load("C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\BERT-SQuAD-master\\model\\pytorch_model.bin",map_location="cpu")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at ..\caffe2\serialize\inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at ..\caffe2\serialize\inline_container.cc:132)
(no backtrace available)

In [6]:
from transformers import (WEIGHTS_NAME, XLNetConfig,
                                  XLNetForQuestionAnswering, XLNetTokenizer)

In [7]:
MODEL_NAME = 'xlnet-large-cased' 
config = XLNetConfig.from_pretrained(MODEL_NAME)

In [13]:
tokenizer = XLNetTokenizer.from_pretrained(MODEL_NAME)
model = XLNetForQuestionAnswering.from_pretrained(MODEL_NAME)

KeyboardInterrupt: 

In [ ]:
model.eval()
model.load_state_dict(torch.load("../input/savedbertmodelandtokenizer/model/pytorch_model.bin",map_location=torch.device('cpu')) ,strict=False)

In [ ]:
model.load_state_dict(torch.load("../input/savedbertmodelandtokenizer/model/pytorch_model.bin"

In [ ]:
tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
        model = XLMForQuestionAnswering.from_pretrained('xlnet-large-cased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
        start_positions = torch.tensor([1])
        end_positions = torch.tensor([3])
        outputs = model(input_ids, start_positions=start_positions, end_positions=end_positions)
        loss, start_scores, end_scores = outputs[:2]

In [1]:
from bert import QA

ModuleNotFoundError: No module named 'bert'

In [8]:
from bert import QA

model = QA("C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion")

doc = ("Scope defines the  process will be applicable for all referrals hired through the employee referral program, effective 1st October 2021.It covers all open positions in the Organization that would fall under Employee Referral Programme as announced by the Talent Acquisition Team and as approved by the Management.Special referral schemes catering to critical needs as announced by the Talent Acquisition Team and as approved by the management.")

q = ('Do yo have special referral schemes ?')

answer = model.predict(doc,q)

print(answer['answer'])
# 1975
print(answer.keys())

empty
dict_keys(['answer', 'start', 'end', 'confidence', 'document'])


In [6]:
python $SQUAD_DIR/evaluate-v1.1.py dev-v1.1.json ../models/wwm_uncased_finetuned_squad/predictions.json
{"exact_match": 86.91579943235573, "f1": 93.1532499015869}

SyntaxError: invalid syntax (Temp/ipykernel_3512/504374866.py, line 1)

In [ ]:
with open(r"C:\\Users\\JaiGatiri\\Downloads\\test-er.json", "r") as read_file:
    test = json.load(read_file)

In [8]:
model = "C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\BERT-SQuAD-master\\model"

In [7]:
!pip install simpletransformers

     -------------------------------------- 250.7/250.7 kB 3.1 MB/s eta 0:00:00
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sentencepiece-0.1.99-cp310-cp310-win_amd64.whl (977 kB)
     ---------------------------------------- 5.6/5.6 MB 10.8 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 10.1 MB/s eta 0:00:00
     ------------------------------------- 486.2/486.2 kB 10.1 MB/s eta 0:00:00
     ------------------------------------- 214.7/214.7 kB 12.8 MB/s eta 0:00:00
  Using cached pathtools-0.1.2.tar.gz (11 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
     -------------------------------------- 134.3/134.3 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 93.9/93.9 kB ? eta 0:00:00
     -------------


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Evaluate the model
result, texts = model.eval_model(test)

AttributeError: 'str' object has no attribute 'eval_model'

In [ ]:
python $SQUAD_DIR/evaluate-v1.1.py $SQUAD_DIR/dev-v1.1.json ../models/wwm_uncased_finetuned_squad/predictions.json
{"exact_match": 86.91579943235573, "f1": 93.1532499015869}

In [9]:
!pip uninstall torch==1.2.0+cpu torchvision==0.4.0+cpu -f https://download.pytorch.org/whl/torch_stable.html


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

no such option: -f


In [3]:
pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.6.0%2Bcpu-cp37-cp37m-win_amd64.whl (167.7 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.7.0%2Bcpu-cp37-cp37m-win_amd64.whl (362 kB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.4.1 requires Pillow<8.3.0, but you have pillow 9.2.0 which is incompatible.

[notice] A new release of pip available: 22.3 -> 23.2
[notice] To update, run: c:\users\jaigatiri\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip


In [5]:
%cd c:\users\jaigatiri\appdata\local\programs\python\python310\lib\site-packages

c:\users\jaigatiri\appdata\local\programs\python\python310\lib\site-packages


In [12]:
!python -m pip install transformers


[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import torch

In [14]:
torch. __version__

'1.6.0+cpu'

In [8]:
model_path = 'C:/Users/JaiGatiri/Documents/cdqa-pred/cdQA/squad-2.0-model'

In [18]:
cd C:\Users\JaiGatiri\Documents\cdqa-pred\BERT-SQuAD-master-jai\model-oldversion\

C:\Users\JaiGatiri\Documents\cdqa-pred\BERT-SQuAD-master-jai\model-oldversion


In [19]:
from bert import QA
from pathlib import Path

model = QA("model-oldversion")

doc = "GENERAL ADMINISTRATIVE MATTERS sections includes Working hours which describes The work hours of the Employee shall be from 09:00 AM to 06:30 PM for General Shift.The COMPANY shall have the authority to change the time schedule and shall be intimated to the Employee in advance.Working hours shorter than 04:00 shall be considered as Half Day for the purpose of wages."
q = 'what are the work hours of employees?'

answer = model.predict(doc,q)

print(answer['answer'])
# 1975
print(answer.keys())

ModuleNotFoundError: No module named 'pytorch_transformers'

In [10]:
%cd C:\Users\JaiGatiri\Documents\cdqa-pred\BERT-SQuAD-master-jai\transformers

C:\Users\JaiGatiri\Documents\cdqa-pred\BERT-SQuAD-master-jai\transformers


In [9]:
pip install -e .

Obtaining file:///C:/Users/JaiGatiri/Downloads/BERT-SQuAD-master/transformers
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0.dev0
    Uninstalling transformers-4.28.0.dev0:
      Successfully uninstalled transformers-4.28.0.dev0
  Running setup.py develop for transformers


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.1.3 requires transformers<4.13.0,>=3.4.0, but you have transformers 2.1.1 which is incompatible.
simpletransformers 0.63.9 requires transformers>=4.6.0, but you have transformers 2.1.1 which is incompatible.
You should consider upgrading via the 'e:\entity_extraction_combined_model\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
export SQUAD_DIR=C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master

SyntaxError: invalid syntax (Temp/ipykernel_15284/675483100.py, line 1)

In [1]:
%cd C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\transformers

C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\transformers


In [26]:
python_version()

NameError: name 'python_version' is not defined

In [3]:
from platform import python_version

In [4]:
python_version()

'3.7.9'

In [12]:
!python C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\transformers\examples\run_squad.py \
    --model_type xlnet \
    --model_name_or_path xlnet-large-cased \
    --do_train \
    --do_eval \
    --do_lower_case \
    --train_file C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\all_3docs_annotn_cdqa.json \
    --predict_file C:\Users\JaiGatiri\Downloa1s\BERT-SQuAD-master\test-er.json \
    --learning_rate 3e-5 \
    --num_train_epochs 1 \
    --max_seq_length 384 \
    --doc_stride 128 \
    --output_dir C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\wwm_cased_finetuned_squad \
    --per_gpu_eval_batch_size  \
    --per_gpu_train_batch_size=64

^C


In [1]:
%cd "C:\Users\JaiGatiri\Downloads\transformers-main\"

C:\Users\JaiGatiri\Downloads\transformers-main


In [15]:
!pip install -e .

Obtaining file:///C:/Users/JaiGatiri/Downloads/transformers-main
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for transformers (pyproject.toml): started
  Building editable for transformers (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.28.0.dev0-0.editable-py3-none-any.whl size=34850 sha256=e7fb5f1cdd1140f3792cd9bb388969b0abe920b629d172808e2e7202f287b219
  Stored in directory: C:\Users\JAIGAT~1\AppData\Local\Temp\pip-ephem-wheel-cache-4tljuda0\wheels\b5\06\50\94ba15c

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.1.3 requires transformers<4.13.0,>=3.4.0, but you have transformers 4.28.0.dev0 which is incompatible.
You should consider upgrading via the 'e:\entity_extraction_combined_model\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [3]:
from transformers import (
    XLNetConfig,
    XLNetForQuestionAnswering,
    XLNetTokenizer,
    squad_convert_examples_to_features)

In [4]:
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

In [13]:
import os
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    XLNetConfig,
    XLNetForQuestionAnswering,
    XLNetTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

# READER NOTE: Set this flag to use own model, or use pretrained model in the Hugging Face repository
use_own_model = True

if use_own_model:
  model_name_or_path = "C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion"
else:
  model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"

output_dir = ""

# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

def to_list(tensor):
    return tensor.detach().cpu().tolist()

# Setup model
config_class, model_class, tokenizer_class = (
    XLNetConfig, XLNetForQuestionAnswering, XLNetTokenizer)
config = config_class.from_pretrained(model_name_or_path)
tokenizer = tokenizer_class.from_pretrained(
    model_name_or_path, do_lower_case=True)
model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

ImportError: cannot import name 'squad_convert_examples_to_features' from 'transformers' (c:\users\jaigatiri\downloads\bert-squad-master\transformers\transformers\__init__.py)

In [ ]:
"
q = 'Are the referral bonus programme  is applicable  only  for  permanent  employee ?'

In [12]:
context = "All employees who are active at the time of the employee referral bonus payment date - both referrer and the referee will need to on rolls with Techversant and not serving the notice period.All who are directly involved in external recruitment are not eligible The  referral  bonus  programme  is applicable  only  for  permanent  employee  hires  and  fixed  term consultants. If the referral candidate joins as contractors, the referrer will be eligible for referral bonus only upon their successful conversion to regular rolls of the company.To  be  eligible  for  referral  bonus,  the  profile  has  to  be  sent  to  the  Talent  Acquisition  team  by  the employee at the mentioned email id or portal as applicable.  Any application processed without the involvement of the Talent Acquisition team will not be eligible for a referral bonus.In the event a referral is received through multiple referrers, the first referrer becomes eligible for thereferral  bonus.  The  Talent  Acquisition  Team  shall  intimate  the  other  referrers  before  initiating  the recruitment process.Referral bonus will not be applicable for fresher recruitment.Referral  bonus  will  not be  applicable  for  positions  under  the  Enabling  /  Support  Functions,unless there is an exception as approved by the Management."
questions = ["Are the referral bonus programme  is applicable  only  for  permanent  employee ?"]

# Run method
predictions = run_prediction(questions, context)

# Print results
for key in predictions.keys():
  print(predictions[key])

NameError: name 'run_prediction' is not defined

In [14]:
from transformers import XLNetConfig,XLNetForQuestionAnswering, XLNetTokenizer

In [15]:
model_path = 'C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion'
model = XLNetForQuestionAnswering.from_pretrained(model_path)
tokenizer = XLNetTokenizer.from_pretrained(model_path)

In [6]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')

In [2]:
model = "C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion"

In [16]:
def get_prediction(context, question):
  inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)
  outputs = model(**inputs)
  
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 
  
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
  
  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)
  def white_space_fix(text):
    return " ".join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match(prediction, truth):
    return bool(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  return round(2 * (prec * rec) / (prec + rec), 2)
  
def question_answer(context, question,answer):
  prediction = get_prediction(context,question)
  em_score = exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f'Question: {question}')
  print(f'Prediction: {prediction}')
  print(f'True Answer: {answer}')
  print(f'Exact match: {em_score}')
  print(f'F1 score: {f1_score}\n')

In [17]:
context = """Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special considerations may be required."""


questions = ["Does the guideline change ?"]

answers = ["Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization."]

for question, answer in zip(questions, answers):
  question_answer(context, question, answer)

AttributeError: 'dict' object has no attribute 'to'

In [35]:
import transformers
from transformers import XLNetConfig,XLNetForQuestionAnswering, XLNetTokenizer
import torch
import numpy as np

class QAPipe():
    def __init__(self,context):
        #self.nlp = pipeline("question-answering")
        self.context = context
        self.input_ids = None
        self.text_tokens = None
        self.outputs = None
        self.segment_ids = None
        self.tokenizer = XLNetTokenizer.from_pretrained("C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion")
        self.model = XLNetTokenizer.from_pretrained("C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion")
    def __init_tokenizer__(self):
        self.tokenizer = XLNetTokenizer.from_pretrained("C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion")
    
    def __init_model__(self):
        self.model = XLNetForQuestionAnswering.from_pretrained("C:\\Users\\JaiGatiri\\Downloads\\BERT-SQuAD-master\\model-oldversion")
     
    def get_ids(self):
        return self.input_ids
        
        
    def generate_text_from_token(self):
        return self.tokenizer.convert_ids_to_tokens(self.input_ids)

    def get_output(self,question):
        print("enter the function")
        self.input_ids = self.tokenizer.encode(question,self.context,max_length=2015,truncation=True)
        tokens = self.generate_text_from_token()
        SEP_index = self.input_ids.index(102) #Getting index of first SEP token
        len_question = SEP_index+1
        len_answer = len(self.input_ids)-len_question
        segment_ids = [0]*len_question + [1]*len_answer #Segment Ids will 0 for question and 1 for answer
        self.segment_ids = segment_ids
        #Getting start and end scores for answer and converting input arrays to tensor before passing to the module
        start_token_scores = self.model(torch.tensor([self.input_ids]),token_type_ids=torch.tensor([segment_ids]))[0]
        end_token_scores = self.model(torch.tensor([self.input_ids]),token_type_ids=torch.tensor([segment_ids]))[1]
        #Converting scores tensor to numpy arrays so that we can use numpy functions
        start_token_scores =start_token_scores.detach().numpy().flatten()
        end_token_scores = end_token_scores.detach().numpy().flatten()
        #picking start and end index of an answer based on start/end indices with highest scores
        answer_start_index = np.argmax(start_token_scores)
        answer_end_index = np.argmax(end_token_scores)
         #Getting Scores for start and end token of the answer. 
        start_token_score = np.round(start_token_scores[answer_start_index],2)
        end_token_score = np.round(end_token_scores[answer_end_index],2)
        #Cleaning answer text
        
        answer = tokens[answer_start_index]
        for i in range(answer_start_index+1,answer_end_index+1):
            if tokens[i][0:2] == '##':
                answer+=tokens[i][2:]
            else:
                answer+= ' '+tokens[i]
        if(answer_start_index ==0) or (start_token_score<0) or (answer == '[SEP]') or (answer_end_index < answer_start_index):
            answer = "Sorry!, Not able to answer your question. Try a different question related to the context."
        print(answer_start_index,answer_end_index,start_token_score,end_token_score,start_token_scores,end_token_scores,answer)
        return (answer_start_index,answer_end_index,start_token_score,end_token_score,start_token_scores,end_token_scores,answer)

In [36]:
QAPipe(context = """Purpose and objective  of  the  Employee  Referral  Programme  process  document  is  to  provide guidelines necessary to assist all  employees at Techversant who refer talent profiles to the Organization. This program  is  designed  to  help  source  talent, build  diversity,  and  find  the  right  candidates  for  open positions  and  to  reward  our  employees  for  the  right  referrals. This  programme  will  encourage  all  at Techversant to act as talent ambassadors for the Organization.Techversant is an equal opportunity employer with a diverse workforce.All referred candidates  will be given  the  same  consideration  and  will  pass  through  the  established  process  and  procedures  of  the Organization.Please note that it is not the intent for these  guidelines to  describe every situation. Management judgment will / may be necessary in certain situations which are not addressed here,or where special considerations may be required.""")

In [13]:
%cd C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\transformers

C:\Users\JaiGatiri\Downloads\BERT-SQuAD-master\transformers


In [17]:
pip install -e.

Obtaining file:///C:/Users/JaiGatiri/Downloads/BERT-SQuAD-master/transformers
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: transformers
    Found existing installation: transformers 2.1.1
    Uninstalling transformers-2.1.1:
      Successfully uninstalled transformers-2.1.1
  Running setup.py develop for transformers
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.1.3 requires transformers<4.13.0,>=3.4.0, but you have transformers 2.1.1 which is incompatible.
simpletransformers 0.63.9 requires transformers>=4.6.0, but you have transformers 2.1.1 which is incompatible.
You should consider upgrading via the 'e:\entity_extraction_combined_model\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [21]:
import os
import torch
import time
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import transformers

In [22]:
from transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForQuestionAnswering, BertTokenizer,
                                  XLMConfig, XLMForQuestionAnswering,
                                  XLMTokenizer, XLNetConfig,
                                  XLNetForQuestionAnswering,
                                  XLNetTokenizer,
                                  DistilBertConfig, DistilBertForQuestionAnswering, DistilBertTokenizer)